In [7]:
df = spark.read.format("csv")\
	.option("header", "true")\
    .option("inferSchema", "true")\
    .load("C:/data/retail-data/by-day/2010-12-01.csv")


In [8]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [10]:
df.createOrReplaceTempView("dfTable")

In [11]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [13]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365).select("InvoiceNo", "Description").show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [20]:
df.where("InvoiceNo = 536365").show(5, False)

df.where("InvoiceNo <> 536365").show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [16]:
from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600


In [17]:
priceFilter 

Column<'(UnitPrice > 600)'>

In [18]:
descripFilter = instr(df.Description, "POSTAGE") >= 1

In [21]:
df.where(df.StockCode.isin("DOT")).where(priceFilter & descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      NULL|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [22]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [23]:
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [24]:
from pyspark.sql.functions import initcap
df.select(col("Description"), initcap(col("Description"))).show()

+--------------------+--------------------+
|         Description|initcap(Description)|
+--------------------+--------------------+
|WHITE HANGING HEA...|White Hanging Hea...|
| WHITE METAL LANTERN| White Metal Lantern|
|CREAM CUPID HEART...|Cream Cupid Heart...|
|KNITTED UNION FLA...|Knitted Union Fla...|
|RED WOOLLY HOTTIE...|Red Woolly Hottie...|
|SET 7 BABUSHKA NE...|Set 7 Babushka Ne...|
|GLASS STAR FROSTE...|Glass Star Froste...|
|HAND WARMER UNION...|Hand Warmer Union...|
|HAND WARMER RED P...|Hand Warmer Red P...|
|ASSORTED COLOUR B...|Assorted Colour B...|
|POPPY'S PLAYHOUSE...|Poppy's Playhouse...|
|POPPY'S PLAYHOUSE...|Poppy's Playhouse...|
|FELTCRAFT PRINCES...|Feltcraft Princes...|
|IVORY KNITTED MUG...|Ivory Knitted Mug...|
|BOX OF 6 ASSORTED...|Box Of 6 Assorted...|
|BOX OF VINTAGE JI...|Box Of Vintage Ji...|
|BOX OF VINTAGE AL...|Box Of Vintage Al...|
|HOME BUILDING BLO...|Home Building Blo...|
|LOVE BUILDING BLO...|Love Building Blo...|
|RECIPE BOX WITH M...|Recipe Box

In [25]:
from pyspark.sql.functions import lower, upper
df.select(col("Description"), lower(col("Description")), upper(lower(col("Description")))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



In [26]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit("  HELLO  ")).alias("ltrim"),
rtrim(lit("  HELLO  ")).alias("rtrim"),
trim(lit("  HELLO  ")).alias("trim"),
lpad(lit("HELLO"), 3, " ").alias("lp"),
rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+-------+-------+-----+---+----------+
|  ltrim|  rtrim| trim| lp|        rp|
+-------+-------+-----+---+----------+
|HELLO  |  HELLO|HELLO|HEL|HELLO     |
|HELLO  |  HELLO|HELLO|HEL|HELLO     |
+-------+-------+-----+---+----------+
only showing top 2 rows



In [27]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
	.withColumn("today", current_date())\
	.withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")

dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [28]:
dateDF.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2024-06-20|2024-06-20 22:19:...|
|  1|2024-06-20|2024-06-20 22:19:...|
|  2|2024-06-20|2024-06-20 22:19:...|
|  3|2024-06-20|2024-06-20 22:19:...|
|  4|2024-06-20|2024-06-20 22:19:...|
|  5|2024-06-20|2024-06-20 22:19:...|
|  6|2024-06-20|2024-06-20 22:19:...|
|  7|2024-06-20|2024-06-20 22:19:...|
|  8|2024-06-20|2024-06-20 22:19:...|
|  9|2024-06-20|2024-06-20 22:19:...|
+---+----------+--------------------+



In [29]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2024-06-15|        2024-06-25|
+------------------+------------------+
only showing top 1 row



In [30]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
	.select(datediff(col("week_ago"), col("today"))).show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [31]:
dateDF.select(
      to_date(lit("2016-01-01")).alias("start"),
      to_date(lit("2017-05-22")).alias("end"))\
      .select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [32]:
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               NULL|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



In [33]:
from pyspark.sql.functions import to_date


dateFormat = "yyyy-dd-MM"

cleanDateDF = spark.range(1).select(
  to_date(lit("2017-12-11"), dateFormat).alias("date"),
  to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [34]:
cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [35]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))

complexDF.createOrReplaceTempView("complexDF")

In [36]:
complexDF.show()

+--------------------+
|             complex|
+--------------------+
|{WHITE HANGING HE...|
|{WHITE METAL LANT...|
|{CREAM CUPID HEAR...|
|{KNITTED UNION FL...|
|{RED WOOLLY HOTTI...|
|{SET 7 BABUSHKA N...|
|{GLASS STAR FROST...|
|{HAND WARMER UNIO...|
|{HAND WARMER RED ...|
|{ASSORTED COLOUR ...|
|{POPPY'S PLAYHOUS...|
|{POPPY'S PLAYHOUS...|
|{FELTCRAFT PRINCE...|
|{IVORY KNITTED MU...|
|{BOX OF 6 ASSORTE...|
|{BOX OF VINTAGE J...|
|{BOX OF VINTAGE A...|
|{HOME BUILDING BL...|
|{LOVE BUILDING BL...|
|{RECIPE BOX WITH ...|
+--------------------+
only showing top 20 rows



In [37]:
complexDF.select("complex.InvoiceNo").show()

+---------+
|InvoiceNo|
+---------+
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536365|
|   536366|
|   536366|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
|   536367|
+---------+
only showing top 20 rows



In [38]:
complexDF.select(col("complex").getField("Description")).show()

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|CREAM CUPID HEART...|
|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|
|SET 7 BABUSHKA NE...|
|GLASS STAR FROSTE...|
|HAND WARMER UNION...|
|HAND WARMER RED P...|
|ASSORTED COLOUR B...|
|POPPY'S PLAYHOUSE...|
|POPPY'S PLAYHOUSE...|
|FELTCRAFT PRINCES...|
|IVORY KNITTED MUG...|
|BOX OF 6 ASSORTED...|
|BOX OF VINTAGE JI...|
|BOX OF VINTAGE AL...|
|HOME BUILDING BLO...|
|LOVE BUILDING BLO...|
|RECIPE BOX WITH M...|
+--------------------+
only showing top 20 rows



In [39]:
from pyspark.sql.functions import split


df.select(split(col("Description"), " ")).show(20)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
|     [CREAM, CUPID, HE...|
|     [KNITTED, UNION, ...|
|     [RED, WOOLLY, HOT...|
|     [SET, 7, BABUSHKA...|
|     [GLASS, STAR, FRO...|
|     [HAND, WARMER, UN...|
|     [HAND, WARMER, RE...|
|     [ASSORTED, COLOUR...|
|     [POPPY'S, PLAYHOU...|
|     [POPPY'S, PLAYHOU...|
|     [FELTCRAFT, PRINC...|
|     [IVORY, KNITTED, ...|
|     [BOX, OF, 6, ASSO...|
|     [BOX, OF, VINTAGE...|
|     [BOX, OF, VINTAGE...|
|     [HOME, BUILDING, ...|
|     [LOVE, BUILDING, ...|
|     [RECIPE, BOX, WIT...|
+-------------------------+
only showing top 20 rows



In [40]:
df.select(split(col("Description"), " ").alias("array_col")).selectExpr("array_col[0]","array_col[1]").show()

+------------+------------+
|array_col[0]|array_col[1]|
+------------+------------+
|       WHITE|     HANGING|
|       WHITE|       METAL|
|       CREAM|       CUPID|
|     KNITTED|       UNION|
|         RED|      WOOLLY|
|         SET|           7|
|       GLASS|        STAR|
|        HAND|      WARMER|
|        HAND|      WARMER|
|    ASSORTED|      COLOUR|
|     POPPY'S|   PLAYHOUSE|
|     POPPY'S|   PLAYHOUSE|
|   FELTCRAFT|    PRINCESS|
|       IVORY|     KNITTED|
|         BOX|          OF|
|         BOX|          OF|
|         BOX|          OF|
|        HOME|    BUILDING|
|        LOVE|    BUILDING|
|      RECIPE|         BOX|
+------------+------------+
only showing top 20 rows



In [41]:
from pyspark.sql.functions import explode

df.withColumn("splitted", split(col("Description"), " "))\
	.withColumn("exploded", explode(col("splitted")))\
    .select("Description", "InvoiceNo", "exploded").show(2)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
+--------------------+---------+--------+
only showing top 2 rows



In [42]:
jsonDF = spark.range(1).selectExpr("""'{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [43]:
jsonDF.show()

+--------------------+
|          jsonString|
+--------------------+
|{"myJSONKey" : {"...|
+--------------------+



In [44]:
udfExampleDF = spark.range(5).toDF("num")

def power3(double_value):
    return double_value ** 3

power3(2.0)

8.0

In [45]:
from pyspark.sql.functions import udf
power3udf = udf(power3)